## Cleaning Data

### About dataset

The dataset contains more than 5000 rows and 3 columns; features, location and price of house, the features column contains details about house such as number of bedroom, bathrooms, parking lot, the size of the land, etc,. So the idea is create more columns with each detail of features columns, verify if exists missing values, change datatypes, etc.

In [21]:
import numpy as np
import pandas as pd

**Get data**

In [2]:
houses = pd.read_csv('./houses.csv')

**Review of Data**

In [3]:
houses.shape

(5860, 3)

In [4]:
# info about data
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5860 entries, 0 to 5859
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   features  5860 non-null   object
 1   location  5860 non-null   object
 2   price     5860 non-null   object
dtypes: object(3)
memory usage: 137.5+ KB


In [5]:
houses.head(1)

,features,location,price
0,207 m² 135 m² 3 hab. 2 baños 2 estac.,"Los Chillos, Quito",USD 132.000


**First I am going to separate the features column in different columns**

In [6]:
separate_feature = houses.features.str.split(' ', n=15, expand=True)
separate_feature.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,207,m²,,135,m²,,3,hab.,,2,baños,,2,estac.,


In [7]:
houses['landsize'] = separate_feature[1]
houses['housesize'] = separate_feature[4]
houses['bedroom'] = separate_feature[7]
houses['bathroom'] = separate_feature[10]
houses['parkinglot'] = separate_feature[13]

In [8]:
houses.head(1)

,features,location,price,landsize,housesize,bedroom,bathroom,parkinglot
0,207 m² 135 m² 3 hab. 2 baños 2 estac.,"Los Chillos, Quito",USD 132.000,207,135,3,2,2


**From location column get the neighborhood name**

In [9]:
houses['neighborhood'] = houses['location'].str.split(',').str[0].str.strip()

In [10]:
houses.head(1)

,features,location,price,landsize,housesize,bedroom,bathroom,parkinglot,neighborhood
0,207 m² 135 m² 3 hab. 2 baños 2 estac.,"Los Chillos, Quito",USD 132.000,207,135,3,2,2,Los Chillos


**Cleaaning price column, I want to delete the USD string from this column and delete the point into the numbers**

In [11]:
houses['price'] = houses['price'].str.split(' ').str[1].str.strip().str.replace('.', '', regex=True)

In [12]:
new_df= houses[['neighborhood', 'landsize', 'housesize', 'bedroom', 'bathroom',
'parkinglot', 'price']]

In [13]:
new_df.head(1)

,neighborhood,landsize,housesize,bedroom,bathroom,parkinglot,price
0,Los Chillos,207,135,3,2,2,132000


**Verifying if exists missing values**

In [14]:
new_df.isna().sum() / new_df.shape[0] * 100

neighborhood     0.000000
landsize         0.000000
housesize        0.017065
bedroom          0.392491
bathroom         1.058020
parkinglot      12.354949
price            0.000000
dtype: float64

Ok, the dataset have some missing values, the most missing values are in parkinglot column

In [15]:
new_df = new_df.dropna(axis=0)

In [16]:
new_df.isna().sum()

neighborhood    0
landsize        0
housesize       0
bedroom         0
bathroom        0
parkinglot      0
price           0
dtype: int64

**Transform object datatype to int**

In [17]:
new_df['landsize'] = new_df['landsize'].astype('int64')
new_df['housesize'] = new_df['housesize'].astype('int64')
new_df['bedroom'] = new_df['bedroom'].astype('int64')
new_df['bathroom'] = new_df['bathroom'].astype('int64')
new_df['parkinglot'] = new_df['parkinglot'].astype('int64')
new_df['price'] = new_df['price'].astype('int64')

In [18]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5136 entries, 0 to 5859
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   neighborhood  5136 non-null   object
 1   landsize      5136 non-null   int64 
 2   housesize     5136 non-null   int64 
 3   bedroom       5136 non-null   int64 
 4   bathroom      5136 non-null   int64 
 5   parkinglot    5136 non-null   int64 
 6   price         5136 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 321.0+ KB


**Outliers Detection**

In [27]:
# method to recognize outlier
def detect_outliers(df, feature):
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75) 
    IQR = Q3 - Q1
    upper_value = Q3 + 1.5 * IQR
    lower_value = Q1 - 1.5 * IQR
    outliers_index = df.index[ (df[feature] < lower_value) | (df[feature] > upper_value) ]
    return outliers_index

In [30]:
# empty list to extract all indexes of outliers
index_list = []
for feature in ['landsize', 'housesize', 'bedroom', 'bathroom', 'parkinglot', 'price']:
    index_list.extend( detect_outliers(new_df, feature) )

In [37]:
# function to remove ouliers
def remove_outliers(df, indexe):
    indexes = sorted( set(index_list) ) # drop the duplicate values in index_list and sorted
    df = df.drop(indexes)
    return df

In [38]:
df_cleaned = remove_outliers(new_df, index_list)

In [39]:
df_cleaned

,neighborhood,landsize,housesize,bedroom,bathroom,parkinglot,price
0,Los Chillos,207,135,3,2,2,132000
1,Mitad del Mundo,125,80,3,2,1,58000
2,Ponceano,130,130,3,2,2,110000
5,Conocoto,193,143,3,2,2,79000
6,Norte de Quito,170,150,3,2,2,145000
...,...,...,...,...,...,...,...
5852,Norte de Quito,145,140,4,2,1,124099
5854,Cumbayá,540,540,4,6,4,450000
5857,Los Chillos,196,136,3,2,2,99900
5858,Calderón,200,200,3,2,3,149000


In [41]:
df_cleaned.to_csv('clean_data.csv', index=False)